In [1]:
get_ipython().system('pip install wandb')
get_ipython().system('pip install onnxruntime')
get_ipython().system('pip install -q gradio')

You should consider upgrading via the '/Users/abhisheksarkar/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/abhisheksarkar/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/abhisheksarkar/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr
os.environ["WANDB_API_KEY"] = "526145026d9030cfd5d66bc0f200786831a5b80a"
run = wandb.init(project='auto_loan')
ARTIFACT_NAME = 'XGBoost_AutoLoan:v1'
artifact = run.use_artifact('abhsark/autoloanprod/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()
artifact_dir

'./artifacts/XGBoost_AutoLoan:v1'

In [29]:
import onnx
model = onnx.load('./artifacts/autoloan_xgboost.onnx')
output =[node.name for node in model.graph.output]

input_all = [node.name for node in model.graph.input]
input_initializer =  [node.name for node in model.graph.initializer]
net_feed_input = list(set(input_all)  - set(input_initializer))

print('Inputs: ', net_feed_input)
print('Outputs: ', output)

Inputs:  ['Car_Owned', 'Client_Education', 'Registration_Days', 'Client_Gender', 'Child_Count', 'Client_Occupation', 'Workphone_Working', 'House_Own', 'Credit_Amount', 'Client_Income_Type', 'Active_Loan', 'Cleint_City_Rating', 'Type_Organization']
Outputs:  ['output_label', 'output_probability']


In [5]:

x_columns = ['Client_Gender','Client_Occupation','Car_Owned','Client_Education','Active_Loan','Cleint_City_Rating','Workphone_Working',
             'House_Own','Type_Organization','Client_Income_Type','Registration_Days','Child_Count','Credit_Amount']


cat_features = ['Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own',
                'Mobile_Tag', 'Homephone_Tag', 'Workphone_Working', 'Cleint_City_Rating']

num_features = list(set(x_columns) - set(cat_features))
num_features



['Client_Education',
 'Registration_Days',
 'Client_Gender',
 'Child_Count',
 'Client_Occupation',
 'Credit_Amount',
 'Client_Income_Type',
 'Type_Organization']

In [30]:

def predict_price(clientGender, clientOccupation, carOwn,
                  clientEdu, activeLoan, clientCity, 
                  workPhone, ownHouse, orgType, 
                  incomeType, regDays, children,amntNeed):

  
    inputs_dict = {'Client_Gender' : float(clientGender), 
              'Client_Occupation': float(clientOccupation), 
              'Car_Owned': float(carOwn), 
              'Client_Education': float(clientEdu), 
              'Active_Loan': float(activeLoan), 
              'Cleint_City_Rating': float(clientCity), 
              'Workphone_Working': float(workPhone), 
              'House_Own': float(ownHouse), 
              'Type_Organization': float(orgType), 
              'Client_Income_Type': float(incomeType), 
              'Registration_Days': float(regDays), 
              'Child_Count': float(children),
              'Credit_Amount': float(amntNeed)}
    

    df = pd.DataFrame(inputs_dict, index = [0])
    print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))            
  
    sess = rt.InferenceSession(artifact_dir + '/autoloan_xgboost.onnx')
    pred_onx = sess.run(None, inputs)

    predicted_price = float(pred_onx[0][0,0])
    return predicted_price
    
    #return {f'Expected sale price of the car is: INR {np.round(3, 2)} lakhs' }

In [31]:
predict_price(clientGender=float(2),
                          clientOccupation=3,
                          carOwn=5,
                          clientEdu=float(2), 
                          activeLoan=2, 
                          clientCity=4, 
                          workPhone=5, 
                          ownHouse=1,
                          orgType=55, 
                          incomeType=1000.0, 
                          regDays=185.0, 
                          children=4,
                          amntNeed=12312)
                          
                          

   Client_Gender  Client_Occupation  Car_Owned  Client_Education  Active_Loan  \
0            2.0                3.0        5.0               2.0          2.0   

   Cleint_City_Rating  Workphone_Working  House_Own  Type_Organization  \
0                 4.0                5.0        1.0               55.0   

   Client_Income_Type  Registration_Days  Child_Count  Credit_Amount  
0              1000.0              185.0          4.0        12312.0  


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(double)) , expected: (tensor(float))